In [ ]:
# COPY THIS
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/data102')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

[sodapy](https://github.com/xmunoz/sodapy) installation & example

In [ ]:
!pip install sodapy
from sodapy import Socrata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


## U.S. Chronic Disease Indicators (CDI)

https://chronicdata.cdc.gov/Chronic-Disease-Indicators/U-S-Chronic-Disease-Indicators-CDI-/g4ie-h725

### From CSV

In [ ]:
cdi = pd.read_csv('data/cdi.csv')
cdi.columns

<ipython-input-18-d3bea021a8b3>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  cdi = pd.read_csv('data/cdi.csv')


Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DatavalueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'GeoLocation', 'ResponseID', 'LocationID', 'TopicID', 'QuestionID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')

In [ ]:
cdi[['Longitude', 'Latitude']] = cdi['GeoLocation'].str.replace("[()]", "", regex=True).str.split(" ", expand=True)[[1, 2]].astype(float)

In [ ]:
# columns of interest?
cdi_filtered = cdi[
    np.isin(cdi['Topic'], ["Chronic Obstructive Pulmonary Disease", "Asthma", "Cardiovascular Disease", "Tobacco"])
]
cdi_filtered.shape

(450362, 36)

In [ ]:
cdi['Question'].unique()

array(['Hospitalizations for asthma', 'Asthma mortality rate',
       'Cancer of the oral cavity and pharynx, mortality',
       'Cancer of the prostate, mortality',
       'Invasive cancer (all sites combined), mortality',
       'Cancer of the female breast, mortality',
       'Cancer of the female cervix, mortality',
       'Cancer of the colon and rectum (colorectal), mortality',
       'Cancer of the lung and bronchus, mortality',
       'Invasive melanoma, incidence', 'Melanoma, mortality',
       'Mortality with end-stage renal disease',
       'Hospitalization for chronic obstructive pulmonary disease as first-listed diagnosis',
       'Hospitalization for chronic obstructive pulmonary disease as any diagnosis',
       'Hospitalization for chronic obstructive pulmonary disease as first-listed diagnosis among Medicare-eligible persons aged >= 65 years',
       'Hospitalization for chronic obstructive pulmonary disease as any diagnosis among Medicare-eligible persons aged >= 65 y

In [ ]:
cdi_filtered.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3,Longitude,Latitude
0,2014,2014,AR,Arkansas,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,-92.274491,34.748650
1,2018,2018,CO,Colorado,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN,-106.133611,38.843841
2,2018,2018,DC,District of Columbia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN,-77.036871,38.907192
3,2017,2017,GA,Georgia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN,-83.627580,32.839681
4,2010,2010,MI,Michigan,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,AST3_1,NMBR,RACE,HIS,NaN,NaN,NaN,NaN,-84.714390,44.661320


### Using CDC API

In [ ]:
topics = [
  "COPD", # Chronic Obstructive Pulmonary Disease
  "AST", # Asthma
  "CVD", # Cardiovascular Disease
  "TOB" # Tobacco
]

In [ ]:
client = Socrata("chronicdata.cdc.gov", None)
results = client.get("g4ie-h725", where="topicid = 'COPD'", limit=100000) # `where` is SQL-style
cdi = pd.DataFrame.from_records(results)
cdi.columns

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'topic', 'question', 'datavaluetype', 'datavalue', 'datavaluealt',
       'stratificationcategory1', 'stratification1', 'geolocation',
       'locationid', 'topicid', 'questionid', 'datavaluetypeid',
       'stratificationcategoryid1', 'stratificationid1',
       'datavaluefootnotesymbol', 'datavaluefootnote', 'datavalueunit',
       'lowconfidencelimit', 'highconfidencelimit'],
      dtype='object')

In [ ]:
# geo = cdi.geolocation.map(lambda x: x['coordinates'] if (type(x) is dict) else [np.nan, np.nan])
# cdi[['Longitude', 'Latitude']] = geo.tolist()

In [ ]:
cdi['datavalue'] = cdi['datavalue'].astype(float)

In [ ]:
cdi.head()

,yearstart,yearend,locationabbr,locationdesc,datasource,topic,question,datavaluetype,datavalue,datavaluealt,...,topicid,questionid,datavaluetypeid,stratificationcategoryid1,stratificationid1,datavaluefootnotesymbol,datavaluefootnote,datavalueunit,lowconfidencelimit,highconfidencelimit
0,2016,2016,OR,Oregon,SEDD; SID,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,Number,4097.0,4097,...,COPD,COPD5_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN,NaN
1,2017,2017,OR,Oregon,SEDD; SID,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,Number,2281.0,2281,...,COPD,COPD5_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,NaN
2,2018,2018,PR,Puerto Rico,SEDD; SID,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,Number,NaN,NaN,...,COPD,COPD5_1,NMBR,OVERALL,OVR,-,No data available,NaN,NaN,NaN
3,2017,2017,SD,South Dakota,SEDD; SID,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,Number,1015.0,1015,...,COPD,COPD5_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN,NaN
4,2015,2015,TX,Texas,SEDD; SID,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,Number,NaN,NaN,...,COPD,COPD5_1,NMBR,GENDER,GENF,-,No data available,NaN,NaN,NaN


In [ ]:
state_prevalance = cdi[cdi['datavaluetypeid'] == 'AGEADJPREV']
state_prevalance = state_prevalance.groupby('locationabbr').agg({'datavalue':np.nanmean}) #, 'Longitude':'first', 'Latitude':'first'})
state_prevalance.head()

,datavalue
locationabbr,
AK,38.484000
AL,41.612500
AR,42.432479
AZ,40.290813
CA,40.640541


In [ ]:
import plotly.graph_objects as go

# fig = go.Figure(
#   data=go.Scattergeo(
#     lon = state_prevalance['Longitude'],
#     lat = state_prevalance['Latitude'],
#     mode = 'markers',
#     marker_color = state_prevalance['datavalue'],
#     text = np.round(state_prevalance['datavalue'], 2)
#   ),
# )

# fig.update_layout(xaxis={'fixedrange':True}, yaxis={'fixedrange':True}) # doesn't work?

# fig.show()

In [ ]:
state_prevalance = state_prevalance.drop(["VI", "GU", "PR"], axis=0)

In [ ]:
import plotly.express as px
fig = px.choropleth(
    locations=state_prevalance.index,
    locationmode="USA-states",
    color=state_prevalance['datavalue'],
    scope="usa",
)
fig.show()

## Daily Census Tract-Level PM2.5 Concentrations, 2011-2014

https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4

This dataset and the Ozone one are very big (> 8 GB), so we can download smaller amounts of data from the CDC website using an API called sodapy.

I did download all the data but it takes pandas forever to read all the rows. You can read the first $n$ rows using argument `nrows`, but this only reads the top rows and you can't filter for specific rows.

In [ ]:
# pm25 = pd.read_csv('data/pm2_5.csv', nrows=20000)
# pm25.head()

,year,date,statefips,countyfips,ctfips,latitude,longitude,ds_o3_pred,ds_o3_stdd
0,2013,03AUG2013,47,47095,47095960200,36.26229,-89.53672,40.5599,6.4340
1,2013,03AUG2013,47,47165,47165021103,36.26238,-86.62209,37.5527,4.3740
2,2013,03AUG2013,37,37193,37193960100,36.26316,-80.94678,38.7162,5.2400
3,2013,03AUG2013,40,40103,40103957000,36.26445,-97.28429,36.4844,5.5102
4,2013,03AUG2013,32,32003,32003003232,36.26472,-115.30277,60.7359,5.4093


In [ ]:
# API code example from https://dev.socrata.com/foundry/data.cdc.gov/fcqm-xrf4
from sodapy import Socrata

client = Socrata("data.cdc.gov", None)

# arguments are SQL-style: see https://dev.socrata.com/docs/queries/
results = client.get("fcqm-xrf4", where="date = '01JAN2014'", limit=100000)
pm25 = pd.DataFrame.from_records(results)
pm25

,year,date,statefips,countyfips,ctfips,latitude,longitude,ds_pm_pred,ds_pm_stdd
0,2014,01JAN2014,12,12087,12087972600,24.54941,-81.79179,9.9932,9.6032
1,2014,01JAN2014,12,12087,12087972400,24.55265,-81.79997,9.8096,9.5746
2,2014,01JAN2014,12,12087,12087972300,24.55602,-81.79279,9.7349,9.1891
3,2014,01JAN2014,12,12087,12087972000,24.55621,-81.76469,9.6942,9.0292
4,2014,01JAN2014,12,12087,12087972200,24.55726,-81.78251,9.6435,9.454
...,...,...,...,...,...,...,...,...,...
72278,2014,01JAN2014,53,53073,53073010403,48.95131,-122.77596,20.8583,15.2178
72279,2014,01JAN2014,53,53073,53073010301,48.9554,-122.53016,16.756,11.5158
72280,2014,01JAN2014,53,53073,53073010401,48.97138,-122.66349,19.8669,13.5748
72281,2014,01JAN2014,53,53073,53073010303,48.97593,-122.43313,14.579,10.2366


In [ ]:
pm25['ds_pm_pred'] = pm25['ds_pm_pred'].astype(float)
pm25['ds_pm_stdd'] = pm25['ds_pm_stdd'].astype(float)

In [ ]:
county_pm25 = pm25.groupby('countyfips').agg({"ds_pm_pred": np.mean, "latitude": np.median, "longitude": np.median})

In [ ]:
# counties geojson data download
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
county_pm25['county'] = county_pm25.index.astype(str).str.zfill(5)
county_pm25['sqrt_ds_pm_pred'] = np.sqrt(county_pm25['ds_pm_pred'])
county_pm25.head()

,ds_pm_pred,latitude,longitude,county,sqrt_ds_pm_pred,log_ds_pm_pred
countyfips,,,,,,
10001,11.867881,39.134715,-75.543715,10001,3.444979,2.473836
10003,16.112748,39.726390,-75.610755,10003,4.014069,2.779611
10005,11.817547,38.646260,-75.254290,10005,3.437666,2.469585
1001,20.942633,32.473145,-86.484390,01001,4.576312,3.041787
1003,20.020926,30.535920,-87.768040,01003,4.474475,2.996778


In [ ]:
fig = px.choropleth(
    county_pm25,
    locations='county',
    color='sqrt_ds_pm_pred',
    geojson=counties,
    scope="usa",
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

## Daily Census Tract-Level Ozone Concentrations, 2011-2014

https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-Ozone-Concentrations-2011/372p-dx3h

In [ ]:
client = Socrata("data.cdc.gov", None)
results = client.get(
  "372p-dx3h",
  where="`year` = '2014' AND `date` LIKE '01%'", # one sample from each month
  select="`countyfips`, `ds_o3_pred`",
  # group="`countyfips`",
  limit=1000000
)

ozone = pd.DataFrame.from_records(results)
len(ozone)

867396

In [ ]:
client = Socrata("data.cdc.gov", None)
results = client.get(
  "372p-dx3h",
  where="`date` = '01JAN2014'",
  limit=100000
)

ozone = pd.DataFrame.from_records(results)
ozone.head()

,year,date,statefips,countyfips,ctfips,latitude,longitude,ds_o3_pred,ds_o3_stdd
0,2014,01JAN2014,12,12087,12087972600,24.54941,-81.79179,35.2535,8.1593
1,2014,01JAN2014,12,12087,12087972400,24.55265,-81.79997,35.0738,8.1476
2,2014,01JAN2014,12,12087,12087972300,24.55602,-81.79279,35.2918,8.0431
3,2014,01JAN2014,12,12087,12087972000,24.55621,-81.76469,35.0958,8.1774
4,2014,01JAN2014,12,12087,12087972200,24.55726,-81.78251,35.3524,8.2237


In [ ]:
ozone['ds_o3_pred'] = ozone['ds_o3_pred'].astype(float)
# ozone['ds_o3_stdd'] = ozone['ds_o3_stdd'].astype(float)
county_ozone = ozone.groupby('countyfips').agg({"ds_o3_pred": np.mean}) #, "latitude": np.median, "longitude": np.median})
county_ozone['county'] = county_ozone.index.astype(str).str.zfill(5) # add leading 0 for 01001, etc
county_ozone.head()

NameError: ignored

In [ ]:
fig = px.choropleth(
    county_ozone,
    locations='county',
    color='ds_o3_pred',
    geojson=counties,
    scope="usa",
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

# Possible Research Questions

* At least one of your techniques should be either Bayesian hierarchical modeling or causal inference.
* You should use the same dataset to answer both questions.

Options:

1. Multiple hypothesis testing / decision making
1. Bayesian Hierarchical Modeling
1. Prediction with GLMs and nonparametric methods
1. Causal Inference

Some sort of correlation between asthma and air quality (PM2.5 or Ozone)?

* causal inference? or GLM?

Correlation between tobacco use and air quality? Or tobacco use and asthma?'

Change in __ over the years?

* GLM

Counties w/ higher than normal asthma/tobacco/aqi

* multiple hypothesis testing